PARSER TO RESCHEDULE THE PROGRAM

In [8]:
class Instruction:
    def __init__(self, opcode, operands):
        self.opcode = opcode
        self.operands = operands
        self.dependencies = set()  # Store data dependencies

class BasicBlock:
    def __init__(self, instructions):
        self.instructions = instructions

class Function:
    def __init__(self, basic_blocks):
        self.basic_blocks = basic_blocks

def parse_assembly_file(filename):
    with open(filename, 'r') as file:
        assembly_code = file.read()
    return assembly_code

def parse_instructions(assembly_code):
    instructions = []
    for line in assembly_code.split('\n'):
        if line.strip() and not line.strip().startswith('.'):
            if ' ' in line:
                opcode, operands = line.split(' ', 1)
            else:
                opcode = line.strip()
                operands = ''  # No operands
            instructions.append(Instruction(opcode, operands))
    return instructions


def detect_dependencies(instructions):
    for i, instr_i in enumerate(instructions):
        for j, instr_j in enumerate(instructions):
            if i != j:
                if any(op in instr_j.operands for op in instr_i.operands):
                    instr_i.dependencies.add(instr_j)

def optimize_ilp(function, fetch_width):
    optimized_instructions = []
    for block in function.basic_blocks:
        scheduled_instructions = []
        current_fetch_width = 0
        for instr in block.instructions:
            # Check if there are dependencies preventing instruction from being scheduled
            if not any(dep in scheduled_instructions for dep in instr.dependencies):
                # Check if we have enough fetch width available
                if current_fetch_width < fetch_width:
                    scheduled_instructions.append(instr)
                    current_fetch_width += 1
                else:
                    # If fetch width is full, look for the first dependent instruction and insert independent instructions before it
                    idx = next((idx for idx, dep in enumerate(scheduled_instructions) if dep in instr.dependencies), None)
                    if idx is not None:
                        # Split the scheduled instructions at the index where the dependent instruction is found
                        pre_dependent_instr = scheduled_instructions[:idx]
                        dependent_instr = scheduled_instructions[idx:]
                        # Add independent instructions to fill the fetch width before the dependent instruction
                        optimized_instructions.extend(pre_dependent_instr)
                        # Reset the scheduled instructions to contain only the dependent instruction
                        scheduled_instructions = dependent_instr
                        current_fetch_width = 1  # Account for the current instruction
                    else:
                        # If no dependent instruction is found, proceed as usual
                        optimized_instructions.extend(scheduled_instructions)
                        scheduled_instructions = [instr]
                        current_fetch_width = 1  # Account for the current instruction
            else:
                # If instruction has dependencies, add it to the end of the list for later scheduling
                scheduled_instructions.append(instr)
        # Append any remaining scheduled instructions after processing the block
        optimized_instructions.extend(scheduled_instructions)
    return optimized_instructions


def perform_loop_unrolling(function):
    for block in function.basic_blocks:
        for i in range(len(block.instructions)):
            instr = block.instructions[i]
            if instr.opcode == "LOOP":
                # Extract loop count from operands
                loop_count = int(instr.operands)
                # Unroll loop by replicating loop body 'loop_count' times
                unrolled_instructions = []
                for j in range(loop_count):
                    # Copy loop body instructions
                    unrolled_instructions.extend(block.instructions[i+1:])
                # Replace loop instruction with unrolled instructions
                block.instructions = block.instructions[:i] + unrolled_instructions
                break  # Break loop after handling the first loop encountered

def write_rescheduled_program(optimized_instructions, output_file):
    with open(output_file, 'w') as file:
        for instr in optimized_instructions:
            file.write(f"{instr.opcode} {instr.operands}\n")

def main(input_file, fetch_width, output_file):
    assembly_code = parse_assembly_file(input_file)
    instructions = parse_instructions(assembly_code)
    detect_dependencies(instructions)
    function = Function([BasicBlock(instructions)])
    optimized_instructions = optimize_ilp(function, fetch_width)
    perform_loop_unrolling(function)
    write_rescheduled_program(optimized_instructions, output_file)

if __name__ == "__main__":
    input_file = "/content/input_mips.s"
    fetch_width = 4  # Example fetch width
    output_file = "rescheduled_mips.s"
    main(input_file, fetch_width, output_file)


SUPERSCALAR PROCESSOR

In [ ]:
class SuperscalarPipeline:
    def __init__(self, fetch_width, pipeline_depth):
        self.fetch_width = fetch_width
        self.pipeline_depth = pipeline_depth
        self.instruction_buffer = []
        self.clock_cycles = 0
        self.register_status = {}  # Track whether a register is ready
        self.reorder_buffer = []  # Reorder buffer for in-order commit
        self.commit_buffer = []  # Buffer for instructions ready to commit
        self.fetch_units = [None] * 4
        self.decode_units = [None] * 4
        self.execute_units = [None] * 4
        self.commit_units = [None] * 4
        self.branch_predictions = {}  # Store branch predictions using 2-bit predictor
        self.flushed = False  # Flag to indicate if pipeline has been flushed

    def fetch(self, instructions):
        for i in range(self.fetch_width):
            if instructions:
                instruction = instructions.pop(0)
                self.instruction_buffer.append(instruction)

                # Check for branch instructions
                if instruction.op == 'BNEZ' or instruction.op == 'BEQZ':
                    # Predict branch outcome based on history or default to not taken
                    prediction = self.branch_predictions.get(instruction, '00')
                    if prediction in ['01', '10']:
                        # Strongly or weakly predicted taken, fetch next instruction
                        instructions.pop(0)
                        self.instruction_buffer.pop(-1)
                    else:
                        # Weakly or strongly predicted not taken, continue fetching
                        pass

    def handle_branch(self, instruction):
        # Perform branch prediction using 2-bit predictor
        if instruction.op == 'BNEZ' or instruction.op == 'BEQZ':
            # Default prediction
            prediction = '00'
            if instruction in self.branch_predictions:
                # Update prediction based on previous outcome
                prediction = self.branch_predictions[instruction]
                if prediction == '01':
                    prediction = '11'
                elif prediction == '00':
                    prediction = '01'
                elif prediction == '10':
                    prediction = '01'
                elif prediction == '11':
                    prediction = '10'
            self.branch_predictions[instruction] = prediction

    def decode(self):
        decoded_instructions = []
        for _ in range(self.fetch_width):
            if self.instruction_buffer:
                instruction = self.instruction_buffer.pop(0)
                self.handle_branch(instruction)
                decoded_instructions.append(instruction)
        return decoded_instructions

    def execute(self):
        executed_instructions = []
        for i, instruction in enumerate(self.reorder_buffer):
            if instruction in self.execute_units:
                continue
            for j, unit in enumerate(self.execute_units):
                if unit is None:
                    self.execute_units[j] = instruction
                    executed_instructions.append(instruction)
                    break
        return executed_instructions

    def writeback(self, executed_instructions):
        for instruction in executed_instructions:
            self.commit_buffer.append(instruction)
            self.reorder_buffer.remove(instruction)

    def commit(self):
        committed_instructions = []
        for i in range(len(self.commit_units)):
            if len(self.commit_buffer) > 0:
                instruction = self.commit_buffer.pop(0)
                committed_instructions.append(instruction)
                self.commit_units[i] = instruction
                self.update_register_status(instruction)
        return committed_instructions

    def simulate(self, instructions):
        while instructions or self.instruction_buffer or self.reorder_buffer or self.commit_buffer:
            if self.flushed:
                # If the pipeline was flushed due to a misprediction, reset it
                self.flushed = False
                self.reorder_buffer = []
                self.commit_buffer = []
                self.clock_cycles += 1
                continue

            print("Clock cycle:", self.clock_cycles)
            print("Instructions in buffer:", len(self.instruction_buffer))
            print("Reorder buffer:", [str(inst) for inst in self.reorder_buffer])
            print("Commit buffer:", [str(inst) for inst in self.commit_buffer])
            print("Fetch units:", [str(inst) if inst else "Empty" for inst in self.fetch_units])
            print("Decode units:", [str(inst) if inst else "Empty" for inst in self.decode_units])
            print("Execute units:", [str(inst) if inst else "Empty" for inst in self.execute_units])
            print("Commit units:", [str(inst) if inst else "Empty" for inst in self.commit_units])
            print("-----------------------------")

            # Commit stage
            committed_instructions = self.commit()

            # Writeback stage
            self.writeback(committed_instructions)

            # Execute stage
            executed_instructions = self.execute()

            # Decode stage
            decoded_instructions = self.decode()

            # Fetch stage
            if instructions:
                self.fetch(instructions)

            # Increment clock cycles
            self.clock_cycles += 1

    def update_register_status(self, instruction):
        # Mark destination register as ready
        if instruction.dest is not None:
            self.register_status[instruction.dest] = True

# Instruction class for representing MIPS instructions
class Instruction:
    def __init__(self, op, dest=None, src1=None, src2=None):
        self.op = op
        self.dest = dest
        self.src1 = src1
        self.src2 = src2

    def __str__(self):
        return f"{self.op} {self.dest}, {self.src1}, {self.src2}"

def parse_mips_file(file_path):
    instructions = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if parts:
                op = parts[0]
                dest = parts[1] if len(parts) > 1 else None
                src1 = parts[2] if len(parts) > 2 else None
                src2 = parts[3] if len(parts) > 3 else None
                instructions.append(Instruction(op, dest, src1, src2))
    return instructions

def main():
    mips_file_path = "/content/rescheduled_mips.s"  # Path to the MIPS input file
    instructions = parse_mips_file(mips_file_path)
    pipeline = SuperscalarPipeline(fetch_width=4, pipeline_depth=6)  # Adjust parameters for superscalar execution
    pipeline.simulate(instructions)
    print("Total clock cycles:", pipeline.clock_cycles)

if __name__ == "__main__":
    main()


Clock cycle: 0
Instructions in buffer: 0
Reorder buffer: []
Commit buffer: []
Fetch units: ['Empty', 'Empty', 'Empty', 'Empty']
Decode units: ['Empty', 'Empty', 'Empty', 'Empty']
Execute units: ['Empty', 'Empty', 'Empty', 'Empty']
Commit units: ['Empty', 'Empty', 'Empty', 'Empty']
-----------------------------
Clock cycle: 1
Instructions in buffer: 4
Reorder buffer: []
Commit buffer: []
Fetch units: ['Empty', 'Empty', 'Empty', 'Empty']
Decode units: ['Empty', 'Empty', 'Empty', 'Empty']
Execute units: ['Empty', 'Empty', 'Empty', 'Empty']
Commit units: ['Empty', 'Empty', 'Empty', 'Empty']
-----------------------------
Clock cycle: 2
Instructions in buffer: 4
Reorder buffer: []
Commit buffer: []
Fetch units: ['Empty', 'Empty', 'Empty', 'Empty']
Decode units: ['Empty', 'Empty', 'Empty', 'Empty']
Execute units: ['Empty', 'Empty', 'Empty', 'Empty']
Commit units: ['Empty', 'Empty', 'Empty', 'Empty']
-----------------------------
Clock cycle: 3
Instructions in buffer: 4
Reorder buffer: []
Comm

#Latest Updated for superScalar

In [2]:
class SuperscalarPipeline:
    def __init__(self, fetch_width, pipeline_depth):
        self.fetch_width = fetch_width
        self.pipeline_depth = pipeline_depth
        self.instruction_buffer = []
        self.clock_cycles = 0
        self.register_status = {}
        self.reorder_buffer = []
        self.commit_buffer = []
        self.fetch_units = [None] * fetch_width
        self.decode_units = [None] * pipeline_depth
        self.execute_units = [None] * pipeline_depth
        self.commit_units = [None] * pipeline_depth
        self.branch_predictions = {}

    def fetch(self, instructions):
        for _ in range(self.fetch_width):
            if instructions:
                instruction = instructions.pop(0)
                self.instruction_buffer.append(instruction)

    def handle_branch(self, instruction):
        if instruction.op == 'BNEZ' or instruction.op == 'BEQZ':
            prediction = self.branch_predictions.get(instruction, '00')
            if prediction in ['01', '10']:
                instructions.pop(0)
                self.instruction_buffer.pop(-1)
            else:
                pass

    def decode(self):
        decoded_instructions = []
        for _ in range(self.pipeline_depth):
            if self.instruction_buffer:
                instruction = self.instruction_buffer.pop(0)
                self.handle_branch(instruction)
                decoded_instructions.append(instruction)
        return decoded_instructions

    def execute(self):
        executed_instructions = []
        for i, instruction in enumerate(self.decode_units):
            if instruction and not self.execute_units[i]:
                self.execute_units[i] = instruction
                executed_instructions.append(instruction)
        return executed_instructions

    def writeback(self, executed_instructions):
        for instruction in executed_instructions:
            self.commit_buffer.append(instruction)
            self.reorder_buffer.remove(instruction)

    def commit(self):
        committed_instructions = []
        for i, instruction in enumerate(self.execute_units):
            if instruction and not self.commit_units[i]:
                committed_instructions.append(instruction)
                self.commit_units[i] = instruction
                self.update_register_status(instruction)
        return committed_instructions

    def simulate(self, instructions):
        while instructions or self.instruction_buffer or self.reorder_buffer or self.commit_buffer:
            print("Clock cycle:", self.clock_cycles)
            print("Instructions in buffer:", len(self.instruction_buffer))
            print("Reorder buffer:", [str(inst) for inst in self.reorder_buffer])
            print("Commit buffer:", [str(inst) for inst in self.commit_buffer])
            print("Fetch units:", [str(inst) if inst else "Empty" for inst in self.fetch_units])
            print("Decode units:", [str(inst) if inst else "Empty" for inst in self.decode_units])
            print("Execute units:", [str(inst) if inst else "Empty" for inst in self.execute_units])
            print("Commit units:", [str(inst) if inst else "Empty" for inst in self.commit_units])
            print("-----------------------------")

            committed_instructions = self.commit()
            self.writeback(committed_instructions)
            executed_instructions = self.execute()
            decoded_instructions = self.decode()
            if instructions:
                self.fetch(instructions)

            self.clock_cycles += 1

    def update_register_status(self, instruction):
        if instruction.dest is not None:
            self.register_status[instruction.dest] = True


class Instruction:
    def __init__(self, op, dest=None, src1=None, src2=None):
        self.op = op
        self.dest = dest
        self.src1 = src1
        self.src2 = src2

    def __str__(self):
        return f"{self.op} {self.dest}, {self.src1}, {self.src2}"


def parse_mips_file(file_path):
    instructions = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if parts:
                op = parts[0]
                dest = parts[1] if len(parts) > 1 else None
                src1 = parts[2] if len(parts) > 2 else None
                src2 = parts[3] if len(parts) > 3 else None
                instructions.append(Instruction(op, dest, src1, src2))
    return instructions


def main():
    mips_file_path = "/content/input_mips.s"  # Path to the MIPS input file
    instructions = parse_mips_file(mips_file_path)
    pipeline = SuperscalarPipeline(fetch_width=4, pipeline_depth=6)
    pipeline.simulate(instructions)
    print("Total clock cycles:", pipeline.clock_cycles)


if __name__ == "__main__":
    main()


Clock cycle: 0
Instructions in buffer: 0
Reorder buffer: []
Commit buffer: []
Fetch units: ['Empty', 'Empty', 'Empty', 'Empty']
Decode units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
Execute units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
Commit units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
-----------------------------
Clock cycle: 1
Instructions in buffer: 4
Reorder buffer: []
Commit buffer: []
Fetch units: ['Empty', 'Empty', 'Empty', 'Empty']
Decode units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
Execute units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
Commit units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
-----------------------------
Clock cycle: 2
Instructions in buffer: 4
Reorder buffer: []
Commit buffer: []
Fetch units: ['Empty', 'Empty', 'Empty', 'Empty']
Decode units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
Execute units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
C

In [10]:
class SuperscalarPipeline:
    def __init__(self, fetch_width, pipeline_depth):
        self.fetch_width = fetch_width
        self.pipeline_depth = pipeline_depth
        self.instruction_buffer = []
        self.clock_cycles = 0
        self.register_status = {}
        self.reorder_buffer = []
        self.commit_buffer = []
        self.fetch_units = [None] * fetch_width
        self.decode_units = [None] * pipeline_depth
        self.execute_units = [None] * pipeline_depth
        self.commit_units = [None] * pipeline_depth
        self.branch_predictions = {}

    def fetch(self, instructions):
        for _ in range(self.fetch_width):
            if instructions:
                instruction = instructions.pop(0)
                self.instruction_buffer.append(instruction)

    def handle_branch(self, instruction):
        if instruction.op == 'BNEZ' or instruction.op == 'BEQZ':
            prediction = self.branch_predictions.get(instruction, '00')
            if prediction in ['01', '10']:
                instructions.pop(0)
                self.instruction_buffer.pop(-1)
            else:
                pass

    def decode(self):
        decoded_instructions = []
        for _ in range(self.pipeline_depth):
            if self.instruction_buffer:
                instruction = self.instruction_buffer.pop(0)
                self.handle_branch(instruction)
                decoded_instructions.append(instruction)
        return decoded_instructions

    def execute(self):
        executed_instructions = []
        for i, instruction in enumerate(self.decode_units):
            if instruction and not self.execute_units[i]:
                self.execute_units[i] = instruction
                executed_instructions.append(instruction)
        return executed_instructions

    def writeback(self, executed_instructions):
        for instruction in executed_instructions:
            self.commit_buffer.append(instruction)
            self.reorder_buffer.remove(instruction)

    def commit(self):
        committed_instructions = []
        for i, instruction in enumerate(self.execute_units):
            if instruction and not self.commit_units[i]:
                committed_instructions.append(instruction)
                self.commit_units[i] = instruction
                self.update_register_status(instruction)
        return committed_instructions

    def simulate(self, instructions):
        while instructions or self.instruction_buffer or self.reorder_buffer or self.commit_buffer:
            print("Clock cycle:", self.clock_cycles)
            print("Instructions in buffer:", len(self.instruction_buffer))
            print("Reorder buffer:", [str(inst) for inst in self.reorder_buffer])
            print("Commit buffer:", [str(inst) for inst in self.commit_buffer])
            print("Fetch units:", [str(inst) if inst else "Empty" for inst in self.fetch_units])
            print("Decode units:", [str(inst) if inst else "Empty" for inst in self.decode_units])
            print("Execute units:", [str(inst) if inst else "Empty" for inst in self.execute_units])
            print("Commit units:", [str(inst) if inst else "Empty" for inst in self.commit_units])
            print("-----------------------------")

            committed_instructions = self.commit()
            self.writeback(committed_instructions)
            executed_instructions = self.execute()
            decoded_instructions = self.decode()
            if instructions:
                self.fetch(instructions)

            self.clock_cycles += 1

    def update_register_status(self, instruction):
        if instruction.dest is not None:
            self.register_status[instruction.dest] = True


class Instruction:
    def __init__(self, op, dest=None, src1=None, src2=None):
        self.op = op
        self.dest = dest
        self.src1 = src1
        self.src2 = src2

    def __str__(self):
        return f"{self.op} {self.dest}, {self.src1}, {self.src2}"


def parse_mips_file(file_path):
    instructions = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if parts:
                op = parts[0]
                dest = parts[1] if len(parts) > 1 else None
                src1 = parts[2] if len(parts) > 2 else None
                src2 = parts[3] if len(parts) > 3 else None
                instructions.append(Instruction(op, dest, src1, src2))
    return instructions


def main():
    mips_file_path = "/content/rescheduled_mips.s"  # Path to the MIPS input file
    instructions = parse_mips_file(mips_file_path)
    pipeline = SuperscalarPipeline(fetch_width=4, pipeline_depth=6)
    pipeline.simulate(instructions)
    print("Total clock cycles:", pipeline.clock_cycles)


if __name__ == "__main__":
    main()


Clock cycle: 0
Instructions in buffer: 0
Reorder buffer: []
Commit buffer: []
Fetch units: ['Empty', 'Empty', 'Empty', 'Empty']
Decode units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
Execute units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
Commit units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
-----------------------------
Clock cycle: 1
Instructions in buffer: 4
Reorder buffer: []
Commit buffer: []
Fetch units: ['Empty', 'Empty', 'Empty', 'Empty']
Decode units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
Execute units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
Commit units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
-----------------------------
Clock cycle: 2
Instructions in buffer: 4
Reorder buffer: []
Commit buffer: []
Fetch units: ['Empty', 'Empty', 'Empty', 'Empty']
Decode units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
Execute units: ['Empty', 'Empty', 'Empty', 'Empty', 'Empty', 'Empty']
C